In [2]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB 262.6 kB/s eta 0:00:36
   ---------------------------------------- 0.0/9.4 MB 262.6 kB/s eta 0:00:36
   ---------------------------------------- 0.0/9.4 MB 196.9 kB/s eta 0:00:48
   ---------------------------------------- 0.1/9.4 MB 374.1 kB/s eta 0:00:25
   ---------------------------------------- 0.1/9.4 MB 385.0 kB/s eta 0:00:25
    --------------------------------------- 0.1/9.4 MB 343.4 kB/s eta 0:00:28
    --------------------------------------- 0.2/9.4 MB 436.9 kB/s eta 0:00:22
    --------------------------------------- 0.2/9.4 MB 529.7 kB/s eta 0:00:18
   - -------------------------------------- 0.2/9.4 MB 502.2 kB/s eta 0:00:19
   - -------------------------------------- 0.3/9.4 MB 597.8 kB/s eta 0:00:16
   -- -------

In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time


# 기본 설정

## 추가 크롤링 - 1. 남성
- 재킷(man_jacket)
- 코트(man_coat)
- 티셔츠 (Tshirt_df)
- 폴로셔츠 (pshirt_df)
- 팬츠 (pants)
- 쇼츠 (short)

In [11]:
# 크롬 드라이버 경로 설정
driver_path = 'C:/Users/wltn4/project/chromedriver.exe'  # chromedriver의 경로

# ChromeService 및 ChromeOptions 설정
chrome_service = ChromeService(executable_path=driver_path)
chrome_options = webdriver.ChromeOptions()

# 웹 드라이버 시작
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

In [12]:
# 로딩되는 시간 주기
driver.implicitly_wait(10)

### 재킷
1054개 크롤링 -> 986개 (레더 수기 제외)

In [88]:
##남성 재킷류 크롤링 (레더 카테고리 제외)
man_jacket = 'https://www.mytheresa.com/kr/ko/men/clothing/jackets?categories=5437&categories=5435&categories=5431&categories=5432&categories=5434&categories=5433&categories=5430'
driver = webdriver.Chrome()
driver.get(man_jacket)

In [89]:
# 데이터를 저장할 리스트
jacket = []

# 총 의류의 개수
total_jackets = 1054
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_jackets = total_jackets - len(jacket)
        if remaining_jackets <= items_per_page:
            items_to_crawl = remaining_jackets
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[5]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                jacket_name = image_element.get_attribute('alt')
                jacket_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                jacket.append({'Name': jacket_name, 'Image URL': jacket_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_jackets <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[6]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
man_jacket = pd.DataFrame(jacket)


ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to c

In [90]:
man_jacket

,Name,Image URL
0,코튼 캔버스 재킷 | Valentino,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Tinto Terra 코튼 블렌드 블루종 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,코튼 블루종 재킷 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,엠브로이더리 스웨트셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 블렌드 필드 재킷 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
1049,Mersey 플리스 베스트 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
1050,Garson 울 블렌드 베스트 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
1051,Butterfly 스트라이프 코튼 실크 셔츠 | Bode,https://img.mytheresa.com/652/738/100/jpeg/cat...
1052,Crofton 후드 다운 재킷 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [91]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
man_jacket = man_jacket[~man_jacket['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
man_jacket.reset_index(drop=True, inplace=True)
man_jacket

,Name,Image URL
0,코튼 캔버스 재킷 | Valentino,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Tinto Terra 코튼 블렌드 블루종 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,코튼 블루종 재킷 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,엠브로이더리 스웨트셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 블렌드 필드 재킷 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
981,Mersey 플리스 베스트 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
982,Garson 울 블렌드 베스트 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
983,Butterfly 스트라이프 코튼 실크 셔츠 | Bode,https://img.mytheresa.com/652/738/100/jpeg/cat...
984,Crofton 후드 다운 재킷 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 코트
184개 -> 181개(레더 코드 수기 삭제)

In [96]:
##남성 코트류 크롤링 
man_coat = 'https://www.mytheresa.com/kr/ko/men/clothing/coats'
driver = webdriver.Chrome()
driver.get(man_coat)

In [97]:
# 데이터를 저장할 리스트
coat = []

# 총 의류의 개수
total_coats = 184
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_coats = total_coats - len(coat)
        if remaining_coats <= items_per_page:
            items_to_crawl = remaining_coats
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                coat_name = image_element.get_attribute('alt')
                coat_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                coat.append({'Name': coat_name, 'Image URL': coat_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_coats <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

    # 데이터를 pandas DataFrame으로 변환
    man_coat = pd.DataFrame(coat)


ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [98]:
man_coat

,Name,Image URL
0,프린트 테크니컬 카 코트 | Dries Van Noten,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,울 카 코트 | Auralee,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,Byron 레인코트 | Herno,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,싱글 브레스티드 카 코트 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,캐시미어 코트 | Giorgio Armani,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
179,U-lock 코튼 트윌 트렌치 코트 | Givenchy,https://img.mytheresa.com/652/738/100/jpeg/cat...
180,Langford 다운 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
181,Pimlico 트렌치코트 | Burberry,https://img.mytheresa.com/652/738/100/jpeg/cat...
182,Expedition 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [99]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
man_coat = man_coat[~man_coat['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
man_coat.reset_index(drop=True, inplace=True)
man_coat

,Name,Image URL
0,프린트 테크니컬 카 코트 | Dries Van Noten,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,울 카 코트 | Auralee,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,Byron 레인코트 | Herno,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,싱글 브레스티드 카 코트 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,캐시미어 코트 | Giorgio Armani,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
176,U-lock 코튼 트윌 트렌치 코트 | Givenchy,https://img.mytheresa.com/652/738/100/jpeg/cat...
177,Langford 다운 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
178,Pimlico 트렌치코트 | Burberry,https://img.mytheresa.com/652/738/100/jpeg/cat...
179,Expedition 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 티셔츠 
589개 -> 587개(레더 티셔츠 수기 삭제)

In [47]:
##남성 티셔츠류 크롤링 
man_tshirt = 'https://www.mytheresa.com/kr/ko/men/clothing/t-shirts'
driver = webdriver.Chrome()
driver.get(man_tshirt)

In [48]:
# 데이터를 저장할 리스트
Tshirt = []

# 총 의류의 개수
total_Tshirts = 589
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_Tshirts = total_Tshirts - len(Tshirt)
        if remaining_Tshirts <= items_per_page:
            items_to_crawl = remaining_Tshirts
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                Tshirt_name = image_element.get_attribute('alt')
                Tshirt_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                Tshirt.append({'Name': Tshirt_name, 'Image URL': Tshirt_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_Tshirts <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
Tshirt_df = pd.DataFrame(Tshirt)

ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [50]:
Tshirt_df

,Name,Image URL
0,저지 티셔츠 | Tom Ford,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,로고 코튼 저지 티셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,프린트 코튼 티셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,Tinto Terra 로고 코튼 저지 티셔츠 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,Compass 코튼 저지 티셔츠 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
584,Tiny Zoo 아플리케 코튼 티셔츠 | Bode,https://img.mytheresa.com/652/738/100/jpeg/cat...
585,Anagram 엠브로이더리 코튼 티셔츠 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
586,Anagram 엠브로이더리 코튼 티셔츠 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
587,Parker 코튼 티셔츠 | Burberry,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [82]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
Tshirt_df = Tshirt_df[~Tshirt_df['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
Tshirt_df.reset_index(drop=True, inplace=True)
Tshirt_df

,Name,Image URL
0,저지 티셔츠 | Tom Ford,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,로고 코튼 저지 티셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,프린트 코튼 티셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,Tinto Terra 로고 코튼 저지 티셔츠 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,Compass 코튼 저지 티셔츠 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
582,Tiny Zoo 아플리케 코튼 티셔츠 | Bode,https://img.mytheresa.com/652/738/100/jpeg/cat...
583,Anagram 엠브로이더리 코튼 티셔츠 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
584,Anagram 엠브로이더리 코튼 티셔츠 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
585,Parker 코튼 티셔츠 | Burberry,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 폴로셔츠
552개

In [51]:
##남성 폴로셔츠류 크롤링 
man_pshirt = 'https://www.mytheresa.com/kr/ko/men/clothing/polo-shirts'
driver = webdriver.Chrome()
driver.get(man_pshirt)

In [52]:
# 데이터를 저장할 리스트
pshirt = []

# 총 의류의 개수
total_pshirts = 552
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_pshirts = total_pshirts - len(pshirt)
        if remaining_pshirts <= items_per_page:
            items_to_crawl = remaining_pshirts
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                pshirt_name = image_element.get_attribute('alt')
                pshirt_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                pshirt.append({'Name': pshirt_name, 'Image URL': pshirt_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_pshirts <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
pshirt_df = pd.DataFrame(pshirt)

ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [53]:
pshirt_df

,Name,Image URL
0,케이블 니트 코튼 폴로 스웨터 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,로고 코튼 피케 폴로 셔츠 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,리브 니트 코튼 폴로 스웨터 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,코튼 폴로 셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 폴로 셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
547,Rino 리브 니트 코튼 블렌드 폴로 셔츠 | Frescobol Carioca,https://img.mytheresa.com/652/738/100/jpeg/cat...
548,Roddy 니트 폴로 셔츠 | Orlebar Brown,https://img.mytheresa.com/652/738/100/jpeg/cat...
549,Roddy 니트 폴로 셔츠 | Orlebar Brown,https://img.mytheresa.com/652/738/100/jpeg/cat...
550,Phoenix 테리 타월 폴로 셔츠 | Vilebrequin,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 팬츠
755개 -> 746개(레더 팬츠 수기 삭제)

In [57]:
##남성 폴로셔츠류 크롤링 
man_pants = 'https://www.mytheresa.com/kr/ko/men/clothing/pants'
driver = webdriver.Chrome()
driver.get(man_pants)

In [58]:
# 데이터를 저장할 리스트
pants = []

# 총 의류의 개수
total_pants = 755
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_pants = total_pants - len(pants)
        if remaining_pants <= items_per_page:
            items_to_crawl = remaining_pants
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                pants_name = image_element.get_attribute('alt')
                pants_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                pants.append({'Name': pants_name, 'Image URL': pants_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_pants <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
pants = pd.DataFrame(pants)


ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [59]:
pants

,Name,Image URL
0,트윌 스트레이트 팬츠 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Rise 울 팬츠 | Wales Bonner,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,리넨 슬림 팬츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,로우라이즈 크롭 와이드 팬츠 | Thom Browne,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,체크 울 시어서커 팬츠 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
750,Pantaflat 코튼 블렌드 슬림 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
751,트위스티드 리넨 치노 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
752,코튼 치노 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
753,Arizona Solaire 리넨 드로스트링 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [80]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
pants = pants[~pants['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
pants.reset_index(drop=True, inplace=True)
pants

,Name,Image URL
0,트윌 스트레이트 팬츠 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Rise 울 팬츠 | Wales Bonner,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,리넨 슬림 팬츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,로우라이즈 크롭 와이드 팬츠 | Thom Browne,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,체크 울 시어서커 팬츠 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
741,Pantaflat 코튼 블렌드 슬림 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
742,트위스티드 리넨 치노 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
743,코튼 치노 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
744,Arizona Solaire 리넨 드로스트링 팬츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 쇼츠
347개 -> 346개(레더 쇼츠 수기 삭제)

In [60]:
##남성 폴로셔츠류 크롤링 
man_shorts = 'https://www.mytheresa.com/kr/ko/men/clothing/shorts'
driver = webdriver.Chrome()
driver.get(man_shorts)

In [61]:
# 데이터를 저장할 리스트
short = []

# 총 의류의 개수
total_shorts = 347
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_shorts = total_shorts - len(short)
        if remaining_shorts <= items_per_page:
            items_to_crawl = remaining_shorts
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                short_name = image_element.get_attribute('alt')
                short_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                short.append({'Name': short_name, 'Image URL': short_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_shorts <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
short = pd.DataFrame(short)

ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [62]:
short

,Name,Image URL
0,스트라이프 버뮤다 쇼츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,코튼 블렌드 버뮤다 쇼츠 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,트윌 버뮤다 쇼츠 | Ami Paris,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,코튼 크레이프 버뮤다 쇼츠 | Ami Paris,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 카고 쇼츠 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
342,Schooner 스윔 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
343,Coulisse Bermuda 스트라이프 울 실크 블렌드 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
344,Bay 스트라이프 리넨 스윔 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
345,André Arizona Solaire 리넨 버뮤다 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [79]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
short = short[~short['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
short.reset_index(drop=True, inplace=True)
short

,Name,Image URL
0,스트라이프 버뮤다 쇼츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,코튼 블렌드 버뮤다 쇼츠 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,트윌 버뮤다 쇼츠 | Ami Paris,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,코튼 크레이프 버뮤다 쇼츠 | Ami Paris,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 카고 쇼츠 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
341,Schooner 스윔 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
342,Coulisse Bermuda 스트라이프 울 실크 블렌드 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
343,Bay 스트라이프 리넨 스윔 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
344,André Arizona Solaire 리넨 버뮤다 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...


## 추가 크롤링 - 2. 여성
- 상의(블라우스, 셔츠, 티셔츠만) (top)
- 재킷(보머 재킷만) (btjacket)
- 코트 전체 (w_coat)

### 상의(블라우스, 셔츠, 티셔츠 카테고리만)
1654개 -> 1644개(레더 상의 수기 삭제)

In [65]:
##여성 상의류 크롤링 (블라우스, 셔츠, 티셔츠 카테고리만 체크)
women_top = 'https://www.mytheresa.com/kr/ko/women/clothing/tops?categories=0013&categories=0642&categories=0632'
driver = webdriver.Chrome()
driver.get(women_top)

In [66]:
# 데이터를 저장할 리스트
top = []

# 총 의류의 개수
total_tops = 1654
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_tops = total_tops - len(top)
        if remaining_tops <= items_per_page:
            items_to_crawl = remaining_tops
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[5]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                top_name = image_element.get_attribute('alt')
                top_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                top.append({'Name': top_name, 'Image URL': top_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_tops <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[6]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
top = pd.DataFrame(top)

ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to c

In [67]:
top

,Name,Image URL
0,Lenny 오버사이즈 티셔츠 | The Frankie Shop,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Paulette 플로럴 코튼 블라우스 | Velvet,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,Calisto 러치드 데님 셔츠 | Veronica Beard,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,프린트 셔츠 | Jacques Wei,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,크롭 코튼 레이스 셔츠 | Isabel Marant,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
1649,BB Pixel 코튼 저지 티셔츠 | Balenciaga,https://img.mytheresa.com/652/738/100/jpeg/cat...
1650,프린트 코튼 저지 티셔츠 | Alexander McQueen,https://img.mytheresa.com/652/738/100/jpeg/cat...
1651,코튼 포플린 페플럼 셔츠 | Alaïa,https://img.mytheresa.com/652/738/100/jpeg/cat...
1652,빈티지 체크 코튼 셔츠 | Burberry,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [78]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
top = top[~top['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
top.reset_index(drop=True, inplace=True)
top

,Name,Image URL
0,Lenny 오버사이즈 티셔츠 | The Frankie Shop,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Paulette 플로럴 코튼 블라우스 | Velvet,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,Calisto 러치드 데님 셔츠 | Veronica Beard,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,프린트 셔츠 | Jacques Wei,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,크롭 코튼 레이스 셔츠 | Isabel Marant,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
1639,BB Pixel 코튼 저지 티셔츠 | Balenciaga,https://img.mytheresa.com/652/738/100/jpeg/cat...
1640,프린트 코튼 저지 티셔츠 | Alexander McQueen,https://img.mytheresa.com/652/738/100/jpeg/cat...
1641,코튼 포플린 페플럼 셔츠 | Alaïa,https://img.mytheresa.com/652/738/100/jpeg/cat...
1642,빈티지 체크 코튼 셔츠 | Burberry,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 여성 재킷 (보머&트랙 카테고리만)
173개 -> 136개(레더 자켓 수기 삭제)

In [70]:
##여성 재킷류 크롤링 (보머&트랙 카테고리만 체크)
women_btjacket = 'https://www.mytheresa.com/kr/ko/women/clothing/jackets/bomber-track-jackets'
driver = webdriver.Chrome()
driver.get(women_btjacket)

In [71]:
# 데이터를 저장할 리스트
btjacket = []

# 총 의류의 개수
total_btjackets = 173
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_btjackets = total_btjackets - len(btjacket)
        if remaining_btjackets <= items_per_page:
            items_to_crawl = remaining_btjackets
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                btjacket_name = image_element.get_attribute('alt')
                btjacket_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                btjacket.append({'Name': btjacket_name, 'Image URL': btjacket_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_btjackets <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
btjacket = pd.DataFrame(btjacket)

ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [72]:
btjacket

,Name,Image URL
0,크롭 레더 보머 재킷 | Wardrobe.NYC,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Onad 오버사이즈 보머 재킷 | Acne Studios,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,Onad 오버사이즈 보머 재킷 | Acne Studios,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,크롭 보머 재킷 | Victoria Beckham,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,Anja 레더 보머 재킷 | The Attico,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
168,크롭 새틴 보머 재킷 | Jean Paul Gaultier,https://img.mytheresa.com/652/738/100/jpeg/cat...
169,글리터 자카드 트랙 재킷 | Palm Angels,https://img.mytheresa.com/652/738/100/jpeg/cat...
170,The Cube Greenh 패딩 재킷 | Max Mara,https://img.mytheresa.com/652/738/100/jpeg/cat...
171,Cleo 오버사이즈 스웨이드 보머 재킷 | Blazé Milano,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [73]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
btjacket = btjacket[~btjacket['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
btjacket.reset_index(drop=True, inplace=True)
btjacket

,Name,Image URL
0,Onad 오버사이즈 보머 재킷 | Acne Studios,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Onad 오버사이즈 보머 재킷 | Acne Studios,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,크롭 보머 재킷 | Victoria Beckham,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,보우 디테일 새틴 보머 재킷 | Nina Ricci,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,보우 디테일 더치스 새틴 보머 재킷 | Nina Ricci,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
131,크롭 새틴 보머 재킷 | Jean Paul Gaultier,https://img.mytheresa.com/652/738/100/jpeg/cat...
132,글리터 자카드 트랙 재킷 | Palm Angels,https://img.mytheresa.com/652/738/100/jpeg/cat...
133,The Cube Greenh 패딩 재킷 | Max Mara,https://img.mytheresa.com/652/738/100/jpeg/cat...
134,Cleo 오버사이즈 스웨이드 보머 재킷 | Blazé Milano,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 코트
582개 -> 539개(레더 코트 수기 삭제)

In [74]:
##여성 코트류 크롤링
women_coat = 'https://www.mytheresa.com/kr/ko/women/clothing/coats'
driver = webdriver.Chrome()
driver.get(women_coat)

In [75]:
# 데이터를 저장할 리스트
w_coat = []

# 총 의류의 개수
total_w_coats = 582
# 페이지당 의류의 수
items_per_page = 60
# 현재 페이지 번호
current_page = 1

try:
    while True:
        # 마지막 페이지인지 확인
        remaining_w_coats = total_w_coats - len(w_coat)
        if remaining_w_coats <= items_per_page:
            items_to_crawl = remaining_w_coats
        else:
            items_to_crawl = items_per_page

        # 현재 페이지의 이미지 정보 수집
        for i in range(1, items_to_crawl + 1):
            xpath = f'//*[@id="root"]/div/div[2]/div[1]/div[4]/div[{i}]/a/div[1]/div[1]/img'
            try:
                # 이미지가 나타날 때까지 기다림 (최대 10초)
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
                image_element = driver.find_element(By.XPATH, xpath)
                w_coat_name = image_element.get_attribute('alt')
                w_coat_img = image_element.get_attribute('src')
                
                # 데이터 리스트에 추가
                w_coat.append({'Name': w_coat_name, 'Image URL': w_coat_img})
            
            except TimeoutException:
                print(f'Timed out waiting for element at xpath: {xpath}')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감
            except NoSuchElementException:
                print(f'Element not found at xpath: {xpath}. Skipping.')
                continue  # 요소를 찾지 못하면 다음 반복으로 넘어감

        # 크롤링할 의류가 더 없으면 종료
        if remaining_w_coats <= items_per_page:
            print(f'All items have been scraped. Stopping scraping.')
            break
        
        # 다음 페이지로 이동
        next_button_xpath = '//*[@id="root"]/div/div[2]/div[1]/div[5]/div/div[3]/a[1]'
        # 다음 페이지 버튼이 있는지 확인
        next_button = driver.find_elements(By.XPATH, next_button_xpath)
        if len(next_button) == 0:
            print(f'No more next button found. Stopping scraping.')
            break
        
        # 버튼이 클릭 가능할 때까지 대기하고 클릭
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        
        # 다음 버튼이 가려져 있을 경우 스크롤하여 가시성 조정
        driver.execute_script("arguments[0].scrollIntoView();", next_button[0])
        time.sleep(1)  # 스크롤 후 대기
        
        # 클릭 시도
        try:
            next_button[0].click()
        except ElementClickInterceptedException:
            print(f'ElementClickInterceptedException: Click intercepted. Trying to click with JS.')
            driver.execute_script("arguments[0].click();", next_button[0])
        
        # 페이지 번호 업데이트
        current_page += 1
        
        # 페이지 로드 대기 (필요시 조정)
        time.sleep(2)

except NoSuchElementException:
    print(f'No more pages available. Stopping scraping.')

finally:
    # 브라우저 종료
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
w_coat = pd.DataFrame(w_coat)


ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
ElementClickInterceptedException: Click intercepted. Trying to click with JS.
All items have been scraped. Stopping scraping.


In [76]:
w_coat

,Name,Image URL
0,테일러드 카멜 헤어 코트 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,어시메트릭 코튼 블렌드 개버딘 트렌치 코트 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,엠브로이더리 데님 코트 | Dries Van Noten,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,Calao 코튼 캔버스 트렌치 코트 | Max Mara,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,데님 코트 | Dries Van Noten,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
577,x Kim 울 블렌드 코트 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
578,레더 트렌치 코트 | Saint Laurent,https://img.mytheresa.com/652/738/100/jpeg/cat...
579,벨티드 레더 코트 | Saint Laurent,https://img.mytheresa.com/652/738/100/jpeg/cat...
580,커브 더블 브레스티드 트윌 코트 | Bottega Veneta,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [77]:
# '레더'라는 단어가 포함된 행을 찾아 삭제
w_coat = w_coat[~w_coat['Name'].str.contains('레더')]

# 인덱스를 다시 설정하여 행 번호가 연속적으로 이어지도록 함
w_coat.reset_index(drop=True, inplace=True)
w_coat

,Name,Image URL
0,테일러드 카멜 헤어 코트 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,어시메트릭 코튼 블렌드 개버딘 트렌치 코트 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,엠브로이더리 데님 코트 | Dries Van Noten,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,Calao 코튼 캔버스 트렌치 코트 | Max Mara,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,데님 코트 | Dries Van Noten,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
534,패딩 코트 | Alaïa,https://img.mytheresa.com/652/738/100/jpeg/cat...
535,Mystique 다운 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
536,GG Supreme 실크 랩 로브 | Gucci,https://img.mytheresa.com/652/738/100/jpeg/cat...
537,x Kim 울 블렌드 코트 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 남성 의류 데이터프레임 합치기

In [100]:
# 데이터 프레임들을 리스트에 담습니다
man_df = [man_jacket, man_coat, Tshirt_df, pshirt_df, pants, short]

# 리스트에 있는 모든 데이터 프레임을 concat 함수를 사용해 합칩니다
man = pd.concat(man_df, ignore_index=True)

man

,Name,Image URL
0,코튼 캔버스 재킷 | Valentino,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Tinto Terra 코튼 블렌드 블루종 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,코튼 블루종 재킷 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,엠브로이더리 스웨트셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 블렌드 필드 재킷 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
3393,Schooner 스윔 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
3394,Coulisse Bermuda 스트라이프 울 실크 블렌드 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
3395,Bay 스트라이프 리넨 스윔 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...
3396,André Arizona Solaire 리넨 버뮤다 쇼츠 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 여성 의류 데이터프레임 합치기

In [102]:
# 데이터 프레임들을 리스트에 담습니다
women_df = [top, btjacket, w_coat]

# 리스트에 있는 모든 데이터 프레임을 concat 함수를 사용해 합칩니다
women = pd.concat(women_df, ignore_index=True)

women

,Name,Image URL
0,Lenny 오버사이즈 티셔츠 | The Frankie Shop,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Paulette 플로럴 코튼 블라우스 | Velvet,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,Calisto 러치드 데님 셔츠 | Veronica Beard,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,프린트 셔츠 | Jacques Wei,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,크롭 코튼 레이스 셔츠 | Isabel Marant,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
2314,패딩 코트 | Alaïa,https://img.mytheresa.com/652/738/100/jpeg/cat...
2315,Mystique 다운 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
2316,GG Supreme 실크 랩 로브 | Gucci,https://img.mytheresa.com/652/738/100/jpeg/cat...
2317,x Kim 울 블렌드 코트 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...


### 최종 데이터프레임 (여성의류+남성의류)

In [103]:
# 데이터 프레임들을 리스트에 담습니다
df_list = [man, women]

# 리스트에 있는 모든 데이터 프레임을 concat 함수를 사용해 합칩니다
df = pd.concat(df_list, ignore_index=True)

df

,Name,Image URL
0,코튼 캔버스 재킷 | Valentino,https://img.mytheresa.com/652/738/100/jpeg/cat...
1,Tinto Terra 코튼 블렌드 블루종 | Stone Island,https://img.mytheresa.com/652/738/100/jpeg/cat...
2,코튼 블루종 재킷 | Loewe,https://img.mytheresa.com/652/738/100/jpeg/cat...
3,엠브로이더리 스웨트셔츠 | Brunello Cucinelli,https://img.mytheresa.com/652/738/100/jpeg/cat...
4,코튼 블렌드 필드 재킷 | Lemaire,https://img.mytheresa.com/652/738/100/jpeg/cat...
...,...,...
5712,패딩 코트 | Alaïa,https://img.mytheresa.com/652/738/100/jpeg/cat...
5713,Mystique 다운 파카 | Canada Goose,https://img.mytheresa.com/652/738/100/jpeg/cat...
5714,GG Supreme 실크 랩 로브 | Gucci,https://img.mytheresa.com/652/738/100/jpeg/cat...
5715,x Kim 울 블렌드 코트 | Dolce&Gabbana,https://img.mytheresa.com/652/738/100/jpeg/cat...


In [104]:
# 엑셀 파일로 저장
df.to_excel('add_dataset.xlsx', index=False)

-> 총 5717개